# PINN RNN POD reduction

Celda para que funcione en Colab

In [5]:
import os
import sys
import IPython

# Detectar si estamos en Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Ruta base
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/ia_thermal_colab"
else:
    BASE_PATH = os.path.expanduser("~/ia_thermal_colab")

DATASETS_PATH = os.path.join(BASE_PATH, "datasets")
MODELS_PATH = os.path.join(BASE_PATH, "models")

os.makedirs(DATASETS_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

print("Modo:", "Colab" if IN_COLAB else "Local")
print("Ruta datasets:", DATASETS_PATH)
print("Ruta modelos:", MODELS_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Modo: Colab
Ruta datasets: /content/drive/MyDrive/ia_thermal_colab/datasets
Ruta modelos: /content/drive/MyDrive/ia_thermal_colab/models


In [6]:
# 🔄 Parámetros del repo
GIT_REPO_URL = "https://github.com/ismaelgallolopez/ia_thermal.git"  # 👈 Cambia esto
REPO_NAME = GIT_REPO_URL.split("/")[-1].replace(".git", "")
CLONE_PATH = os.path.join(BASE_PATH, REPO_NAME)

# 🧬 Clonar el repositorio si no existe ya
if not os.path.exists(CLONE_PATH):
    !git clone {GIT_REPO_URL} {CLONE_PATH}
else:
    print(f"Repositorio ya clonado en: {CLONE_PATH}")

# 📦 Instalar requirements.txt
req_path = os.path.join(CLONE_PATH, "requirements.txt")
if os.path.exists(req_path):
    !pip install -r {req_path}
else:
    print("No se encontró requirements.txt en el repositorio.")

if IN_COLAB:
    print("🔄 Reinicia el entorno para aplicar los cambios...")
    IPython.display.display(IPython.display.Javascript('''google.colab.restartRuntime()'''))


Repositorio ya clonado en: /content/drive/MyDrive/ia_thermal_colab/ia_thermal
🔄 Reinicia el entorno para aplicar los cambios...


<IPython.core.display.Javascript object>

In [7]:
import numpy as np
import time
import datetime
from IPython.display import display, Markdown
import platform
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import seaborn as sns

# import sklearn
# from sklearn.preprocessing import StandardScaler

import torch
# from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, TensorDataset

np.set_printoptions(threshold=sys.maxsize)
torch.set_default_dtype(torch.float32)

# get the directory path of the file
dir_path = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal")

from plot_functions import *
from Physics_Loss import *

if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal/ismaelgallo")

from convlstm import *

sys.path.append('../Convolutional_NN')

if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal/Convolutional_NN")

from Dataset_Class import *

# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

Configuración global de Matplotlib

In [8]:
plt.rcParams.update({
    # 'text.usetex': True,  # Usar LaTeX para el texto (Local)
    'text.usetex': False,  # NO Usar LaTeX para el texto (Colab)
    'font.family': 'serif',  # Fuente serif
    # 'figure.figsize': (10, 6),  # Tamaño de la figura
    'axes.labelsize': 12,  # Tamaño de las etiquetas de los ejes
    'axes.titlesize': 14,  # Tamaño del título
    'legend.fontsize': 12,  # Tamaño de la leyenda
    'xtick.labelsize': 10,  # Tamaño de las etiquetas del eje x
    'ytick.labelsize': 10,  # Tamaño de las etiquetas del eje y
    'axes.grid': True,  # Habilitar la cuadrícula
    'grid.alpha': 0.75,  # Transparencia de la cuadrícula
    'grid.linestyle': '--'  # Estilo de la línea de la cuadrícula
})

# Configuración de Seaborn
sns.set_context('paper')
sns.set_style('whitegrid')

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print('Using device:', device)

Using device: cpu


In [10]:
system_specs = {
    "os": platform.system(),  # e.g. "Linux", "Windows", "Darwin"
    "os_version": platform.version(),
    "machine": platform.machine(),  # e.g. "x86_64"
    "processor": platform.processor(),  # e.g. "Intel64 Family 6 Model 158"
    "python_version": platform.python_version(),
    "device": str(device)
}
if torch.cuda.is_available():
    system_specs["gpu_name"] = torch.cuda.get_device_name(0)
    system_specs["gpu_memory_total_GB"] = round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 2)
    system_specs["cuda_version"] = torch.version.cuda

<a id='section_1'></a>
# PCB solver trasient

In [11]:
sys.path.append('../scripts')

if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal/scripts")

from PCB_solver_tr import PCB_solver_main, PCB_case_1, PCB_case_2

# Dataset import

Dataset hyperparameters

In [12]:
n_train = 1000
n_test = 200
n_val = 50
time_sim = 100 # seconds

batch_size = 30

sequence_length = time_sim+1 # seconds
dt = 1 # seconds
T_init = 298.0 # Kelvin
nodes_side = 13 # number of nodes in one side of the PCB

Dataset extraction

In [13]:
if IN_COLAB:
  dir_path = BASE_PATH

dataset = load_dataset(base_path=dir_path)  # ← carga el dataset base completo (PCB_transient_dataset.pth)
dataset_train = load_trimmed_dataset(base_path=dir_path, dataset_type='train', max_samples=n_train, time_steps_output=sequence_length)
dataset_test = load_trimmed_dataset(base_path=dir_path, dataset_type='test', max_samples=n_test, time_steps_output=sequence_length)
dataset_val = load_trimmed_dataset(base_path=dir_path, dataset_type='val', max_samples=n_val, time_steps_output=sequence_length)

input_train, output_train = prepare_data_for_convlstm(dataset_train, device=device)
input_test, output_test = prepare_data_for_convlstm(dataset_test, device=device)
input_val, output_val = prepare_data_for_convlstm(dataset_val, device=device)

train_loader = DataLoader(TensorDataset(input_train, output_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(input_test, output_test), batch_size=batch_size, shuffle=False)
val_loader = DataLoader(TensorDataset(input_val, output_val), batch_size=batch_size, shuffle=False)

✅ Cargando dataset base desde: /content/drive/MyDrive/ia_thermal_colab/datasets/PCB_transient_dataset.pth


/content/drive/MyDrive/ia_thermal_colab/ia_thermal/Convolutional_NN/Dataset_Class.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(full_path)


✅ Cargando dataset train desde: /content/drive/MyDrive/ia_thermal_colab/datasets/PCB_transient_dataset_train.pth


/content/drive/MyDrive/ia_thermal_colab/ia_thermal/Convolutional_NN/Dataset_Class.py:266: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_dataset = torch.load(full_path)


✅ Cargando dataset test desde: /content/drive/MyDrive/ia_thermal_colab/datasets/PCB_transient_dataset_test.pth
✅ Cargando dataset val desde: /content/drive/MyDrive/ia_thermal_colab/datasets/PCB_transient_dataset_val.pth


# Convolutional LSTM

## Common to all

### Hyperparameters of training

In [14]:
epochs = 500
lr = 1e-2
lrdecay = 0.1
lrdecay_patience = 10
early_stop_patience = 50

hidden_dims = [32] # [64, 32, 16, 8, 16, 32, 64]
num_layers = len(hidden_dims)
kernel_size = [(3,3) for i in range(num_layers)]

## No-physics Convolutional LSTM

### Model definition

In [15]:
# from convlstm import *

class PCB_ConvLSTM(nn.Module):
    def __init__(self, input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13):
        super().__init__()
        self.convlstm = ConvLSTM(input_dim=input_channels,
                                 hidden_dim=hidden_dims,
                                 kernel_size=kernel_size,
                                 num_layers=len(hidden_dims),
                                 batch_first=True,
                                 bias=True,
                                 return_all_layers=False)

        self.decoder = nn.Conv2d(hidden_dims[-1], 1, kernel_size=1)

    def forward(self, x):
        # x: (B, T, C, H, W)
        lstm_out, _ = self.convlstm(x)  # lstm_out[0]: (B, T, hidden_dim, H, W)

        # Apply decoder to each time step
        decoded = [self.decoder(lstm_out[0][:, t]) for t in range(x.size(1))]
        output = torch.stack(decoded, dim=1)  # (B, T, 1, H, W)
        return output

Definición del modelo

In [16]:
model = PCB_ConvLSTM(input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=lrdecay, patience=lrdecay_patience, verbose=True)

# DEBUGGING
# Ensure data is moved to the appropriate device
batch = next(iter(train_loader))
x, y = batch

print(f"x está en: {x.device}")
print(f"y está en: {y.device}")
print(f"model está en: {next(model.parameters()).device}")

x está en: cpu
y está en: cpu
model está en: cpu


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


### Training

Training and saving best model with best parameters.

In [ ]:
train_loss = []
test_loss = []
best_test_loss = np.inf
epochs_without_improvement = 0

kernel_string = f"{kernel_size[0][0]}x{kernel_size[0][1]}"
model_dir = os.path.join(dir_path, 'models', 'ConvLSTM')

if IN_COLAB:
  model_dir = os.path.join(MODELS_PATH, 'ConvLSTM')

os.makedirs(model_dir, exist_ok=True)

# Nombre del archivo con hiperparámetros
filename = f"PCB_ConvLSTM_nt{n_train}_{time_sim}s_lr{lr}_bs{batch_size}_h{num_layers}_k{kernel_string}.pth"

# Ruta completa del modelo
model_path = os.path.join(model_dir, filename)

# Comprobar si el modelo ya existe

if os.path.exists(model_path):
    display(Markdown(f"**❌ El modelo `{filename}` ya existe. Se omite esta celda para evitar sobreescritura.**"))
    # Detiene la ejecución de esta celda sin interrumpir el notebook
    # raise SystemExit

# ruta para el JSON
json_path = model_path.replace('.pth', '.json')

start_time_training = time.time()
start_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    start_time_epoch = time.time()

    # Entrenamiento
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=False):

        optimizer.zero_grad()
        y_hat = model(x)

        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.detach().item()

    epoch_train_loss = total_loss / len(train_loader)

    # Validación
    model.eval()
    total_test_loss = 0.0
    with torch.no_grad():
        for x_test, y_test in test_loader:
            y_pred = model(x_test)
            test = criterion(y_pred, y_test)
            total_test_loss += test.item()

    epoch_test_loss = total_test_loss / len(test_loader)
    test_loss.append(epoch_test_loss)

    # Scheduler update
    scheduler.step(epoch_test_loss)

    # Early stopping check
    if epoch_test_loss < best_test_loss:
        best_test_loss = epoch_test_loss

        # Guardar el modelo
        torch.save(model.state_dict(), model_path)

        elapsed_training = time.time() - start_time_training
        elapsed_minutes = elapsed_training / 60
        current_lr = optimizer.param_groups[0]['lr']


        # Guardar hiperparámetros en JSON
        params = {
            'start_datetime': start_datetime,
            'training_duration_minutes': elapsed_minutes,
            "system_specs": system_specs,
            'hidden_dims': hidden_dims,
            'kernel_size': kernel_string,
            'batch_size': batch_size,
            'lr': lr,
            "scheduler":{
                "type": "ReduceLROnPlateau",
                "factor": lrdecay,
                "patience": lrdecay_patience,
                "final_lr": current_lr
            },
            'early_stop_patience': early_stop_patience,
            'epochs_trained': epoch + 1,
            'best_test_loss': best_test_loss,
            "train_loss": list(map(float, train_loss)),
            "test_loss": list(map(float, test_loss)),
        }

        with open(json_path, 'w') as f:
            json.dump(params, f, indent=4)

        # print(f"✓ Saving model (epoch {epoch+1}) | test_loss improved to {best_test_loss:.6f}")
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        # print(f"No improvement for {epochs_without_improvement} epoch(s)")

    if epochs_without_improvement >= early_stop_patience:
        print(f"⚠️ Early stopping at epoch {epoch+1} — no improvement for {early_stop_patience} epochs.")
        break

    # Estadísticas finales de la época
    elapsed_epoch = time.time() - start_time_epoch
    # print(f"Epoch {epoch+1:3d} | Train Loss: {epoch_train_loss:.6f} | Test Loss: {epoch_test_loss:.6f} | Time: {elapsed_epoch:.2f}s")

print(f"Entrenamiento finalizado en {elapsed_minutes:.2f} minutos.")

**❌ El modelo `PCB_ConvLSTM_nt1000_100s_lr0.01_bs30_h1_k3x3.pth` ya existe. Se omite esta celda para evitar sobreescritura.**

Epoch 1/500 - Training:  44%|████▍     | 15/34 [03:54<04:51, 15.35s/it]

In [ ]:
print("📁 El modelo se está guardando en:", model_path)


Plotting validation loss and train loss

In [ ]:
plot_loss_evolution(train_loss, test_loss)

### Evaluation

In [ ]:
# load the best model
model = PCB_ConvLSTM(input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

with torch.no_grad():
    y_pred = model(input_val)  # (B, T, 1, H, W)
    val_loss = criterion(y_pred, output_val)
    print(f"Test Loss: {val_loss.item():.6f}")

### Plotting results

We are going to plot the temperature evolution in the four nodes corresponding with the heaters

In [ ]:
id_heaters = [(6,3), (3,6), (9,3), (9,9)]

Boundary conditions

In [ ]:
Q_heaters = np.array([1.0, 1.0, 1.0, 1.0])
T_interfaces = np.array([250, 250, 250, 250])
T_env = 250

Actual values

In [ ]:
T, _, _, _ = PCB_case_2(solver = 'transient', display=False, time = time_sim, dt = dt, T_init = T_init, Q_heaters = Q_heaters, T_interfaces = T_interfaces, Tenv = T_env) # heaters in default position
T = T.reshape(T.shape[0], nodes_side, nodes_side) # reshaping the data grid-shape

Predicted values

In [ ]:
input_tensor = dataset.create_input_from_values(Q_heaters, T_interfaces, T_env, sequence_length=101)

output = model(input_tensor)
output_denorm = dataset.denormalize_output(output)
T_pred = output_denorm[0,:,0,:,:].cpu().detach().numpy()

In [ ]:
plot_nodes_evolution(T_pred, T, id_heaters, together=True)

## Physics informed Convolutional LSTM

In [ ]:
# from Physics_Loss import *

### New loss function validation

Validation of the Physics loss function

In [ ]:
# Crear instancia de la clase de pérdida
physics_loss = PhysicsLossTransient().to(device)

# Parámetros
B = 1        # batch size
H = W = 13    # dimensiones espaciales

# Obtener datos reales del solver
T2, _, interfaces2, heaters2 = PCB_case_2(solver='transient', display=False, time=100, dt=1, T_init=298.0)
T = T2.shape[0]      # número de pasos temporales

# Temperatura: [B, T, 1, 13, 13]
T_tensor = torch.tensor(T2, dtype=torch.float32).view(T, 1, H, W).unsqueeze(0).repeat(B, 1, 1, 1, 1).to(device)

interfaces_input = torch.tensor([list(interfaces2.values())], dtype=torch.float32).repeat(B, 1).to(device) # [B, 4]
heaters_input = torch.tensor([list(heaters2.values())], dtype=torch.float32).repeat(B, 1).to(device) # [B, 4]
Tenv = torch.full((B, 1), 250.0).to(device) # [B, 1]

# Calcular la pérdida
loss = physics_loss(
    T_pred=T_tensor,
    T_true=T_tensor,
    heaters_input=heaters_input,
    interfaces_input=interfaces_input,
    Tenv=Tenv
)

# # Mostrar resultado
print(f"Physics loss (esperada ≈ 0): {loss.item():.6e}")

Validation of the boundary loss function

In [ ]:
# Crear una instancia de BoundaryLoss con valores predeterminados
boundary_loss = BoundaryLoss()

# Ejemplo de tensores
interfaces_example = torch.tensor(list(interfaces2.values()), dtype=torch.float32).unsqueeze(0)
T2_reshaped = torch.tensor(T2).view(1, 101, 1, 13, 13).to(device)  # [B, T, 1, H, W]

# Calcular la pérdida
loss = boundary_loss(T2_reshaped, interfaces_example)
print("Pérdida en las interfaces:", loss.item())

Función para extraer las condiciones de contorno de los tensores de los 3 canales

In [ ]:
def extract_boundary_conditions_from_dataset(input_tensor, dataset: PCBDataset, nodes_side=13):
    """
    Extrae las condiciones de contorno originales (desnormalizadas) a partir de un input_tensor y el dataset asociado.
    input_tensor: tensor de forma [batch, sequence_length, 3, nodes_side, nodes_side]
    """
    input_0 = input_tensor[0, 0]  # [3, 13, 13]

    T_interfaces1 = input_0[0]
    Q_heaters1 = input_0[1]
    T_env1 = input_0[2]

    # Extraer los valores originales usando los métodos de desnormalización del dataset
    T_interfaces_raw = torch.tensor([
        T_interfaces1[0, 0],
        T_interfaces1[0, nodes_side - 1],
        T_interfaces1[nodes_side - 1, nodes_side - 1],
        T_interfaces1[nodes_side - 1, 0]
    ], device=input_tensor.device)
    T_interfaces_in = dataset.denormalize_T_interfaces(T_interfaces_raw)

    Q_heaters_raw = torch.tensor([
        Q_heaters1[6, 3],
        Q_heaters1[3, 6],
        Q_heaters1[9, 3],
        Q_heaters1[9, 9]
    ], device=input_tensor.device)
    Q_heaters_in = dataset.denormalize_Q_heaters(Q_heaters_raw)

    T_env_in = dataset.denormalize_T_env(T_env1[0, 0])

    return Q_heaters_in, T_interfaces_in, T_env_in


def extract_all_boundary_conditions(input_tensor, dataset: PCBDataset, nodes_side=13):
    """
    Extrae las condiciones de contorno desnormalizadas de todos los ejemplos del batch.
    Retorna tres listas: Q_heaters_all, T_interfaces_all, T_env_all.
    """
    batch_size = input_tensor.shape[0]
    Q_heaters_all = []
    T_interfaces_all = []
    T_env_all = []

    for i in range(batch_size):
        q, t_int, t_env = extract_boundary_conditions_from_dataset(input_tensor[i:i+1], dataset, nodes_side)
        Q_heaters_all.append(q)
        T_interfaces_all.append(t_int)
        T_env_all.append(t_env)

    Q_heaters_all = torch.stack(Q_heaters_all)       # [batch_size, 4]
    T_interfaces_all = torch.stack(T_interfaces_all) # [batch_size, 4]
    T_env_all = torch.stack(T_env_all)               # [batch_size]

    return Q_heaters_all, T_interfaces_all, T_env_all


### Model definition

Hyperparameters of training

In [ ]:
mse_weight = 1.0
phy_weight = 0.0000
bnd_weight = 0.0000

In [ ]:
# from convlstm import *

class PCB_ConvLSTM_physics(nn.Module):
    def __init__(self, input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13):
        super().__init__()
        self.convlstm = ConvLSTM(input_dim=input_channels,
                                 hidden_dim=hidden_dims,
                                 kernel_size=kernel_size,
                                 num_layers=len(hidden_dims),
                                 batch_first=True,
                                 bias=True,
                                 return_all_layers=False)

        self.decoder = nn.Conv2d(hidden_dims[-1], 1, kernel_size=1)

    def forward(self, x):
        # x: (B, T, C, H, W)
        lstm_out, _ = self.convlstm(x)  # lstm_out[0]: (B, T, hidden_dim, H, W)

        # Apply decoder to each time step
        decoded = [self.decoder(lstm_out[0][:, t]) for t in range(x.size(1))]
        output = torch.stack(decoded, dim=1)  # (B, T, 1, H, W)
        return output

Definición del modelo

In [ ]:
dataset_train.base_dataset.return_bc = True
dataset_test.base_dataset.return_bc = True

train_ds = prepare_data_with_bc(dataset_train, device=device)
train_loader_phy = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

test_ds = prepare_data_with_bc(dataset_test, device=device)
test_loader_phy = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

model = PCB_ConvLSTM_physics(input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13).to(device)

total_loss_fn = TotalLoss(
    mse_weight=mse_weight,
    physics_weight=phy_weight,
    boundary_weight=bnd_weight,
    denormalize_output_fn=dataset.denormalize_output
)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=lrdecay, patience=lrdecay_patience, verbose=True)

### Training

Training and saving best model with best parameters.

In [ ]:
train_loss = []
test_loss = []
loss_mse = []
loss_phy = []
loss_bndry = []

best_test_loss = np.inf
epochs_without_improvement = 0

kernel_string = f"{kernel_size[0][0]}x{kernel_size[0][1]}"
model_dir = os.path.join(dir_path, 'models', 'ConvLSTM')
if IN_COLAB:
  model_dir = os.path.join(MODELS_PATH, 'ConvLSTM')

os.makedirs(model_dir, exist_ok=True)

# Nombre del archivo con hiperparámetros
filename_phy = f"PCB_ConvLSTM_nt{n_train}_{time_sim}s_lr{lr}_bs{batch_size}_h{len(hidden_dims)}_k{kernel_string}_phy_{phy_weight}_bnd{bnd_weight}.pth"

# Ruta completa del modelo
model_path_phy = os.path.join(model_dir, filename_phy)

# Comprobar si el modelo ya existe
if os.path.exists(model_path_phy):
    display(Markdown(f"**❌ El modelo `{filename_phy}` ya existe. Se omite esta celda para evitar sobreescritura.**"))
    # Detiene la ejecución de esta celda sin interrumpir el notebook
    # raise SystemExit

# ruta para el JSON
json_path_phy = model_path_phy.replace('.pth', '.json')

start_time_training = time.time()
start_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for epoch in range(epochs):
    model.train()

    total_loss = 0.0
    total_loss_mse = 0.0
    total_loss_phy = 0.0
    total_loss_bndry = 0.0
    start_time_epoch = time.time()

    # Entrenamiento
    for x, y, bc_all in tqdm(train_loader_phy, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=False):
        optimizer.zero_grad()

        q = bc_all[:, 0:4] # sin normalizar
        t_int = bc_all[:, 4:8] # sin normalizar
        t_env = bc_all[:, 8].unsqueeze(1) # sin normalizar

        q_denorm     = dataset.denormalize_Q_heaters(q)
        t_int_denorm = dataset.denormalize_T_interfaces(t_int)
        t_env_denorm = dataset.denormalize_T_env(t_env)
        y_denorm     = dataset.denormalize_output(y)

        y_hat = model(x) # model prediction
        y_hat_denorm = dataset.denormalize_output(y_hat)

        loss, loss_mse_batch, loss_phys_batch, loss_bdry_batch = total_loss_fn(y_hat, y, q_denorm, t_int_denorm, t_env_denorm)

        loss.backward()
        optimizer.step()
        total_loss += loss.detach().item()
        total_loss_mse += loss_mse_batch.item()
        total_loss_phy += loss_phys_batch.item()
        total_loss_bndry += loss_bdry_batch.item()

    epoch_train_loss = total_loss / len(train_loader_phy)
    epoch_loss_mse = total_loss_mse / len(train_loader_phy)
    epoch_loss_phy = total_loss_phy / len(train_loader_phy)
    epoch_loss_bndry = total_loss_bndry / len(train_loader_phy)

    train_loss.append(epoch_train_loss)
    loss_mse.append(epoch_loss_mse)
    loss_phy.append(epoch_loss_phy)
    loss_bndry.append(epoch_loss_bndry)

    # Validación
    model.eval()
    total_test_loss = 0.0
    with torch.no_grad():
        for x_test, y_test, bc_test in test_loader_phy:

            q = bc_test[:, 0:4]
            t_int = bc_test[:, 4:8]
            t_env = bc_test[:, 8].unsqueeze(1)

            q_denorm     = dataset.denormalize_Q_heaters(q)
            t_int_denorm = dataset.denormalize_T_interfaces(t_int)
            t_env_denorm = dataset.denormalize_T_env(t_env)
            y_denorm     = dataset.denormalize_output(y)
            y_hat_denorm = dataset.denormalize_output(y_hat)

            y_pred = model(x_test)

            test_loss_comb, _, _, _ = total_loss_fn(y_pred, y_test, q_denorm, t_int_denorm, t_env_denorm)
            total_test_loss += test_loss_comb.item()

    epoch_test_loss = total_test_loss / len(test_loader_phy)
    test_loss.append(epoch_test_loss)

    # Scheduler update
    scheduler.step(epoch_test_loss)

    elapsed_minutes = (time.time() - start_time_training) / 60

    # Early stopping check
    if epoch_test_loss < best_test_loss:
        best_test_loss = epoch_test_loss

        # Guardar el modelo
        torch.save(model.state_dict(), model_path_phy)

        current_lr = optimizer.param_groups[0]['lr']

        # Guardar hiperparámetros en JSON
        params = {
            'start_datetime': start_datetime,
            'training_duration_minutes': elapsed_minutes,
            "system_specs": system_specs,
            'hidden_dims': hidden_dims,
            'kernel_size': kernel_string,
            'batch_size': batch_size,
            'lr': lr,
            "scheduler":{
                "type": "ReduceLROnPlateau",
                "factor": lrdecay,
                "patience": lrdecay_patience,
                "final_lr": current_lr
            },
            'early_stop_patience': early_stop_patience,
            'epochs_trained': epoch + 1,
            'best_test_loss': best_test_loss,
            "train_loss": list(map(float, train_loss)),
            "test_loss": list(map(float, test_loss)),
            "physics": {
                "phy_param": phy_weight,
                "bnd_param": bnd_weight,
            }
        }

        with open(json_path_phy, 'w') as f:
            json.dump(params, f, indent=4)

        # print(f"✓ Saving model (epoch {epoch+1}) | test_loss improved to {best_test_loss:.6f}")
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        # print(f"No improvement for {epochs_without_improvement} epoch(s)")

    if epochs_without_improvement >= early_stop_patience:
        print(f"⚠️ Early stopping at epoch {epoch+1} — no improvement for {early_stop_patience} epochs.")
        break

    # Estadísticas finales de la época
    elapsed_epoch = time.time() - start_time_epoch
    # print(f"Epoch {epoch+1:3d} | Train Loss: {epoch_train_loss:.6f} | Test Loss: {epoch_test_loss:.6f} | Time: {elapsed_epoch:.2f}s")

print(f"Entrenamiento finalizado en {elapsed_minutes:.2f} minutos.")

Plotting validation loss and train loss

In [ ]:
plot_loss_evolution(train_loss, test_loss)

In [ ]:
# Ensure loss_phy and loss_bndry are converted to NumPy arrays for element-wise multiplication
loss_mse = np.array(loss_mse)
loss_phy = np.array(loss_phy)
loss_bndry = np.array(loss_bndry)

plt.plot(loss_mse * mse_weight, label=f'MSE Loss*{mse_weight}', color='blue')
plt.plot(loss_phy * phy_weight, label=f'Physics Loss*{phy_weight}', color='orange')
plt.plot(loss_bndry * bnd_weight, label=f'Boundary Loss*{bnd_weight}', color='green')
plt.plot(test_loss, label='Total Loss', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xlim(0, len(train_loss)-1)
plt.yscale('log')
plt.legend()
plt.title('Loss Components')
plt.show()


### Evaluation

In [ ]:
# load the best model
model = PCB_ConvLSTM_physics(input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13).to(device)
model.load_state_dict(torch.load(model_path_phy))
model.eval()

criterion = nn.MSELoss() # para comparar colo con la recostrucción

with torch.no_grad():
    y_pred = model(input_val)  # (B, T, 1, H, W)
    val_loss = criterion(y_pred, output_val)
    print(f"Test Loss: {val_loss.item():.6f}")

### Plotting results

We are going to plot the temperature evolution in the four nodes corresponding with the heaters

In [ ]:
id_heaters = [(6,3), (3,6), (9,3), (9,9)]

Boundary conditions

In [ ]:
Q_heaters = np.array([1.0, 1.0, 1.0, 1.0])
T_interfaces = np.array([250, 250, 250, 250])
T_env = 250

Actual values

In [ ]:
T, _, _, _ = PCB_case_2(solver = 'transient', display=False, time = time_sim, dt = dt, T_init = T_init, Q_heaters = Q_heaters, T_interfaces = T_interfaces, Tenv = T_env) # heaters in default position
T = T.reshape(T.shape[0], nodes_side, nodes_side) # reshaping the data grid-shape

Predicted values

In [ ]:
input_tensor = dataset.create_input_from_values(Q_heaters, T_interfaces, T_env, sequence_length=sequence_length)

output = model(input_tensor)
output_denorm = dataset.denormalize_output(output)
T_pred = output_denorm[0,:,0,:,:].cpu().detach().numpy()

In [ ]:
plot_nodes_evolution(T_pred, T, id_heaters, together=True)

Error en la predicción

In [ ]:
plot_se_map(T_pred, T, time=100, show_pred=True)

Error en la predicción

In [ ]:
plot_se_map(T_pred, T, time=100, show_pred=True)

## Comparison of models

Loading models

In [ ]:
model = PCB_ConvLSTM(input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13).to(device)
model_phy = PCB_ConvLSTM_physics(input_channels=3, hidden_dims=hidden_dims, kernel_size=kernel_size, height=13, width=13).to(device)

model.load_state_dict(torch.load(model_path))
model.to(device).eval()
model_phy.load_state_dict(torch.load(model_path_phy))
model_phy.to(device).eval()
print(f"Modelos cargados en dispositivo {device} y listos para evaluar.")

Generating random data to evaluate the models

In [ ]:
np.random.seed(0)

Q_random = np.random.uniform(0.1, 1.25, 4)
T_interfaces_random = np.random.uniform(260, 310, 4)
T_env_random = np.random.uniform(260, 310)

input_tensor = dataset.create_input_from_values(Q_heaters, T_interfaces, T_env, sequence_length=sequence_length)

T_true = PCB_case_2(solver='transient', display=False, time=100, dt=1, T_init=298.0, Q_heaters=Q_random, T_interfaces=T_interfaces_random, Tenv=T_env_random)[0]
T_true = T_true.reshape(T_true.shape[0], nodes_side, nodes_side) # reshaping the data grid-shape

pred = model(input_tensor).cpu().detach()[0,:,0,:,:]
T_pred = dataset.denormalize_output(torch.tensor(pred)).numpy()

pred_phy = model_phy(input_tensor).cpu().detach()[0,:,0,:,:]
T_pred_phy = dataset.denormalize_output(torch.tensor(pred_phy)).numpy()

Error calculation

In [ ]:
error = T_true - T_pred
error_phy = T_true - T_pred_phy

error_last = error[-1]
error_phy_last = error_phy[-1]

Non-PINN

In [ ]:
plot_prediction_and_error(T_pred, T_true, t=100, cmap='hot', save_as_pdf=False, filename='prediction_and_error')

PINN

In [ ]:
plot_prediction_and_error(T_pred_phy, T_true, t=-1, cmap='hot', save_as_pdf=False, filename='prediction_and_error')

Error plot comparison

In [ ]:
compare_error_maps_2d(error_last, error_phy_last, ("Error sin física", "Error con física"), save_as_pdf=True)